## Introduction

Zebra mussels (Dreissena polymorpha) are the most devastating of all aquatic invasive species. Billions of dollars are spent annually to prevent the spread and mitigate the impacts to infrastructure and ecosystem services caused by Dreissinid mussels (Nelson et al. 2019; Pimental et al., 2005; Sepulveda et al. 2020) Zebra mussels were introduced into the Great Lakes region most likely during the 1980's via ballast discharge and have since shown up as far away as Texas and California. Overland transport via recreational watercraft trailered from infested lakes to new locations is the main vector for the spread of zebra and quagga mussels (Johnson et al. 2001 ). Understanding vectors and pathways of spread are critical for predicting and preventing the expansion of non-indigenous aquatic species (NAS). However, an empirical understanding of how the number of public accesses (boat launches) on a waterbody influences the overall risk for introduction for zebra/quagga mussels is lacking. This script is designed to join filter and join spatial datasets which would otherwise be cumbersome to deal with in a desktop GIS to answer the question, "How does the number of public accesses impact the probability of the presence of zebra/quagga mussels?" 

<img src='./data/ZM_USGS_Map.png' width="600" height="600">

#### Hypothesis:
If zebra mussel presence is correlated with the level of propagule pressure, then the number of public accesses on a waterbody (boat ramps) will impact the likelihood of introduction because the number of public accesses is the mechanism driving the amount of propagule pressure for a waterbody.

#### Null Hypothesis: 
There is no correlation between the occurence of zebra/quagga and the numbers of public accesses.

#### Prediction:
Logistic regression will indicate a correlation between the occurence of zebra/quagga and the numbers of public accesses. E.g., lakes without zebra/quagga mussels with be clustered at the left with few accesses and lakes with zebra/quagga mussels will be clustered on the right of the plot with more accesses.

### Methods
#### Study Area
There are approximately 205,032 lakes in MN with 4144 larger than 0.50 square kms with 3013 public water accesses dispersed across these lakes. Minnesota has a long history of invasion by non-indigenous aquatic species including by zebra mussels in 1989. Zebra mussels now inhabit 167 MN lakes.

#### Study Organism
Zebra mussels (Dreissena polymorpha; Pallas 1769) are native to the Ponto-Caspian and Black-Sea Drainages. These freshwater bivalve are less than 50 mm as adults and attach to hard substrate via byssal threads. Their lifecycle includes a planktonic veliger stage which promotes downstream dispersal. These invaders impact systems by attaching to available substrate, competing for planktonic resources and biofouling particles.  They clog pipes and other infrastructure and cause huge amounts of economic damage annually.


#### Data Sources
The Non-Indigenous Aquatic Species database has existed for more than 30 years and houses temporally correlated georeferenced  records for over 2,000 NAS at the national scale (Fuller P, Neilson M, 2015).  The 439 Zebra mussel records that range from 1989 to 2022 can be retrieved from https://nas.er.usgs.gov. These records are directly imported into this script via the NAS database API.

<img src='./data/ZM_MN_Map.png' width="600" height="600">

The shapefile for MN Lake polygons was retrieved from the USGS TNM Download (v2.0) from https://apps.nationalmap.gov/downloader/#/. The MN Waterbody Shapefile contains 205,032 lake polygons many of which do not have public accesses. It is reduced to lakes with an area greater than 0.50 square km and with at least 1 public access. The direct link to the zip file is provided within the readme file included with this script.

<img src='./data/MW_Waterbody_Map.png' width="600" height="600">

Waterbody public accesses spatial data is provided by the MN Department of Natural Resources (MNDNRC) and was retrieved as a shapefile from https://gisdata.mn.gov/dataset/loc-water-access-sites.

<img src='./data/ZM_Public_Accesses.png' width="600" height="600">

#### Statistical Approach

logistic regression is used because my dependent variable, the presence/absence of zebra mussels is represented by a '1' or '0' respectively. This method of binary classification is useful for identifying non-linear relationships between a single independent variable and the dependent variable. The assumptions that must be met for a linear model are, linearity, no outliers
independence. Further, we need to make sure that we aren't combining the counts for public accesses together for lakes with the same name.  To assure our training data meets these assumptions, this script includes a function to check for outliers and remove them from the analysis.

#### Model Accuracy
To test the accuracy of the model the initial dataset was split into training and testing by randomly selecting 25% of the data to be held for testing.  A confusion matrix is then used to assess how well the model performed when predicting to the testing dataset.  

#### Hypothesis Testing
The regression plot and coefficients will be used to assess the correlation between the presence/absence of zebra mussels and the number of public accesses and the expected increase/decrease in the log odds of our outcome variable for each unit increase, respectively. The null hypothesis will be falsified if the regression plot shows that lakes with higher numbers of public accesses are 1's and those with fewer accesses are 0's.

## Technical Approach

The goal of this script is to test whether the number of public accesses on a waterbody is correlated with the presence of zebra/quagga mussels for MN. This script first calls to the USGS NAS Database API to get occurrence data. A series of helper functions are then used to convert the retrieved data into a geopandas geodataframe. A column is also added for 'Present' with the value of '1' to indicate the presence of zebra/quagga mussels.

The waterbody(lakes) shapefile is then imported and reduced to lakes greater than 0.50 square km. The public accesses shapefile is then imported as a geopandas geodataframe. To improve the accuracy of spatial joins, the CRS for all geodataframes is then converted to projected reference system (3857).

The geopandas sjoin function is then used to spatially the join public accesses shapefile to the lakes shapefile. The leftjoin argument was used which results in a row for each ramp joined to the columns of the associated lakes. One potential pitfall to consider is there are most likely multiple lakes with the same name.  For example, there are at least 3 Wolf Lakes in MN.  To avoid combining the counts for public accesses for these lakes with the same name, the right index which came along with the previous spatial join is renamed to Lake_ID. The groupby function is then applied along with the count function to get the number of public accesses by waterbody. 

We need to get a bit crafty here. All this joining has resulted in the most recent geometry coming from the access points, but we need the lake polygons. Since the Lake ID is equivalent to the index from the original lakes shapefile, we can set the Lake ID as the index and merge to the original waterbody shapefile.  Now that we have lake polygons with the count for public accesses, we need to spatially join the occurence records to this geodataframe. To do so the sjoin function with the leftjoin option is applied.  Once again, this results in duplicate records for some lakes due to the occurence data containing duplicate spatial records from different years.  Here we can simply use geopandas to remove these duplicates. We now have a geodataframe with positive lakes indicated by a 1 in the 'Present' column and the number of public accesses. 

To get the negative lakes, we create a list of the Lake_ID column for the positive lakes and use it to select the rows from the combined waterbodies and accesses count dataframe that do not match the positive Lake IDs. Since, logistic regression can be thrown off by outliers, the standard deviation is used to identify records within the positive and negative lakes dataset where the count for ramps exceeds +- SD * 5.  5 was chosen so only extreme outliers are removed.

A final dataframe is then created by concatenating the positive and negative dataframes.  This final dataset is then cast to a numpy array and the sklearn train_test_split function is used to set up the data for modeling. The sklearn LogisticRegression function is then fit to the training data and the resulting model is used to predict to the testing data. The regression plot is then visualized using the Seaborn replot function. Accuracy scores are produced for both training and testing datasets using the sklearn score function. The sklearn confusion matrix function is then created via the metrics package and the seaborn heatmap function is used to plot the results. 

Code to use the model to predict to a numpy array containing values for ramps between 0 and 100 is also provided. A secondary goal of this script is to be flexible such that the user can ask the same question for other NAS and within other states.  Instructions for modifying the code for a different NAS taxa and state are provided within the README.md.

Run the following cells to repeat these steps and perform the analysis described within the results section below. This script was designed with the user in mind such that the code style, formatting and variable names are consistent, distinctive and meaningful per the recommendations of Wilson et al (2014). 

In [ ]:
#Import required packages
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import requests
from functools import reduce
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import statsmodels
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std

In [ ]:
## Helper Functions to convert USGS NAS Records into geodataframe.
def _get_col_rename(df, dftype):
    """Returns a dictionary of columns to rename based on the dataframe and type('csv' or 'api')"""
    
    # Build a dictionary of column renamings for use in pandas rename function
    renamed_columns = {}
    column_names = list(df.columns)
    lower_columns = [name.lower().replace(' ','').replace('_','') for name in column_names]
    for i in range(len(column_names)):
        renamed_columns[column_names[i]] = lower_columns[i]

    if dftype == 'csv':
        # build csv rename dictionary
        renamed_columns['museumcatno'] = 'museumcatnumber'
        renamed_columns['huc8number']  = 'huc8'
    elif dftype == 'api':
        # build api rename dictionary
        renamed_columns['key']              = 'specimennumber'
        renamed_columns['decimallatitude']  = 'latitude'
        renamed_columns['decimallongitude'] = 'longitude'
        renamed_columns['latlongsource']    = 'source'
        renamed_columns['latlongaccuracy']  = 'accuracy'
    else:
        raise ValueError(f"Dataframe type '{dftype}' invalid - Accepted inputs are 'csv' or 'api'")

    return renamed_columns

In [ ]:
## Helper Functions to convert USGS NAS Records into geodataframe.
def _manage_cols(df, drop_list=[], name_dict={}):
    """Private method for dropping and renaming columns in a dataframe, as well as creating one standard table from two different forms."""

    for colname in drop_list:
        if colname not in df:
            raise ValueError(f"Can't drop column '{colname}' - '{colname}' does not exist in dataframe")
    for colname in list(name_dict.keys()):
        if colname not in df:
            raise ValueError(f"Can't rename '{colname}' to '{name_dict[colname]}' - '{colname}' does not exist in dataframe")
        if colname in drop_list:
            raise ValueError(f"Can't rename '{colname}' to '{name_dict[colname]}' - '{colname}' in drop_list")

    column_names = np.setdiff1d(list(df.columns), list(name_dict.keys()))
    lower_columns = [name.lower().replace(' ','').replace('_','') for name in column_names]
    for i in range(len(column_names)):
        name_dict[column_names[i]] = lower_columns[i]
    
    df = df.drop(drop_list, axis=1)
    df = df.rename(columns=name_dict)
    
    return df

In [ ]:
## Helper Functions to convert USGS NAS Records into geodataframe.
URL_BASE = 'http://nas.er.usgs.gov/api/v2/'


def api_df(species_id, limit, api_key):
    """Returns a pandas dataframe containing records about a species from the NAS database using their API"""
    
    # Check for API key
    if api_key is not None:
        url_request = f"{URL_BASE}/occurrence/search?species_ID={species_id}&api_key={api_key}"
    else:
        url_request = f"{URL_BASE}/occurrence/search?species_ID={species_id}"
    
    # Get dataframe from API request
    request_json = requests.get(url_request, params={'limit':limit}).json()
    api_df = pd.json_normalize(request_json, 'results')
    api_df = _manage_cols(api_df)

    # Add columns that are in a CSV dataframe but not an API dataframe
    api_df['country']      = np.nan
    api_df['drainagename'] = np.nan

    # Rename columns
    renamed_columns = _get_col_rename(api_df, 'api')
    api_df = api_df.rename(columns=renamed_columns)

    # Reorder columns
    cols = list(api_df.columns)
    cols = cols[0:8] + cols[33:34] + cols[8:33] + cols[34:] # country
    cols = cols[0:16] + cols[34:] + cols[16:34] # drainagename
    api_df = api_df[cols]
    
    return api_df

In [ ]:
# Define Species ID and State
species_id = 5
my_state = 'Minnesota'

In [ ]:
# Run API function to get records from NAS Database
NAS_df = api_df(species_id = species_id, limit = 10000, api_key = {"speciesID": species_id})

In [ ]:
# Get only columns we want 
my_data = NAS_df[["commonname", "state", "latitude", "longitude", "year", "status", "accuracy"]]

In [ ]:
# Filter data to get only records for established and accurate sightings.  Also, filter to just MN records.
my_data_fltr = my_data[(my_data['status'] == 'established') & (my_data['accuracy'] == 'Accurate')
& (my_data['state'] == my_state)]  ## Replace with your state if using your own waterbody shapefile

In [ ]:
# Add column for Present and set values for all rows to 1 
my_data_fltr.loc[:,'Present'] = 1

In [ ]:
# Convert csv to GeoDataFrame using lat/long columns to create point geometries
user_data_gdf = gpd.GeoDataFrame(
    my_data_fltr, geometry=gpd.points_from_xy(my_data_fltr.longitude, my_data_fltr.latitude))

In [ ]:
# Fix the crs (for some reason we need to set to a random crs and then convert to the crs we want for this to work)
user_data_gdf.set_crs(4326, inplace = True, allow_override=True)
my_data = user_data_gdf.to_crs(3857)

In [ ]:
# Check your data
my_data

In [ ]:
## Import waterbody shapefile, Get lakes larger than .5 km squared, change to projected crs and slice to just the lake name and geometry
MN_lakes = gpd.read_file('./data/Shape/NHDWaterbody.shp')
lake_mask = MN_lakes['areasqkm'] > .5
lakes_filtered = MN_lakes.loc[lake_mask].to_crs(3857)
lakes_gpd = lakes_filtered[['gnis_name', 'geometry']]

In [ ]:
## Import Public Accesses shapefile, change to projected crs and slice to just the facility name and geometry
access_df = gpd.read_file('./data/MN_accesses.shp')
access_df_crs = access_df.to_crs(3857)
my_accesses = access_df_crs[['FAC_NAME', 'geometry']]

In [ ]:
# Take a look at the data plots
base = lakes_gpd.plot(color = 'white', edgecolor = 'black')
my_data.plot(ax=base, marker='o', color='red', markersize=2)
base2 = lakes_gpd.plot(color = 'white', edgecolor = 'black')
my_accesses.plot(ax=base2, marker='x', color='green', markersize=1)


In [ ]:
#Spatial join Points to polygons/ Need to do by nearest because boat ramps might not intersect lake polygon
#Use left join because we want to keep all the accesses and join them to the corresponding lake.
accesses_with_lakes = gpd.sjoin_nearest(my_accesses, lakes_gpd, 'left', max_distance = 100) 

In [ ]:
#Slice to what we need
# Rename the columns so we can group by right index (Lake_ID) Using the waterbody original index to groupby 
#ensures we aren't combining the count for ramps from lakes with same name.
accesses_with_lakes_df = pd.DataFrame(accesses_with_lakes[['FAC_NAME', 'index_right']]).rename(columns = {'FAC_NAME' : "ramps", 'index_right' : "lake_ID"})

In [ ]:
# Groupby lake ID and count ramps per lake
ramps = pd.DataFrame(accesses_with_lakes_df.groupby(['lake_ID']).count())

In [ ]:
# re-merge the ramps with the polygons 
ramps_df = gpd.GeoDataFrame(ramps.merge(lakes_gpd, left_index=True, right_index=True, how = 'right')).dropna()

In [ ]:
# Join zm points to ramps layer
positive_lakes = gpd.sjoin_nearest(my_data, ramps_df, how='left')

In [ ]:
# Remove duplicate lake records
positive_lakes2 = positive_lakes.drop_duplicates('gnis_name').rename(columns = {'index_right' : "lake_ID"})

In [ ]:
# Slice to the columns we want
pos_lakes = positive_lakes2[['lake_ID', 'ramps']]

In [ ]:
#create a list of the Lake IDs from positive lakes
positive_lake_IDs = pos_lakes['lake_ID']

In [ ]:
# This is for merging on index to get negative lakes
ramps2 = pd.DataFrame(accesses_with_lakes_df.groupby(['lake_ID'], as_index=False).count())

In [ ]:
# get lakes not in positive_lake_IDs list
neg_lakes = ramps2[~ramps2['lake_ID'].isin(positive_lake_IDs)]

In [ ]:
# identify outliers with standard deviation
neg_outliers = neg_lakes.iloc[:,1].to_numpy().reshape(-1, 1)
# seed the random number generator
seed(1)
# generate univariate observations
neg_data = neg_outliers
# calculate summary statistics
neg_data_mean, neg_data_std = mean(neg_data), std(neg_data)
# identify outliers
neg_cut_off = neg_data_std * 5
neg_lower, neg_upper = neg_data_mean - neg_cut_off, neg_data_mean + neg_cut_off
# identify outliers
neg_outliers = [x for x in neg_data if x < neg_lower or x > neg_upper]
print('Identified outliers: %d' % len(neg_outliers))
# remove outliers
neg_outliers_removed = pd.DataFrame([x for x in neg_data if x >= neg_lower and x <= neg_upper])
print('Non-outlier observations: %d' % len(neg_outliers_removed))
neg_outliers_removed['Present'] = 0
neg_outliers_removed.describe()

In [ ]:
pos_outliers = pos_lakes.iloc[:,1].to_numpy().reshape(-1, 1)
# seed the random number generator
seed(1)
pos_data = pos_outliers
# calculate summary statistics
pos_data_mean, pos_data_std = mean(pos_data), std(pos_data)
# identify outliers
pos_cut_off = pos_data_std * 5
pos_lower, pos_upper = pos_data_mean - pos_cut_off, pos_data_mean + pos_cut_off
# identify outliers
pos_outliers = [x for x in pos_data if x < pos_lower or x > pos_upper]
print('Identified outliers: %d' % len(pos_outliers))
# remove outliers
pos_outliers_removed = pd.DataFrame([x for x in pos_data if x >= pos_lower and x <= pos_upper])
print('Non-outlier observations: %d' % len(pos_outliers_removed))
pos_outliers_removed['Present'] = 1
pos_outliers_removed.describe()

In [ ]:
#Concatenate negative and positive dataframes.
training_data = pd.concat([neg_outliers_removed, pos_outliers_removed]).rename(columns = {0 : "ramps"})

In [ ]:
#SLice to the columns we want and convert to numpy array 
my_model_data = training_data[['ramps', 'Present']].to_numpy()

In [ ]:
#Use Scikit Learn function to split model data into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(
my_model_data[:,0].reshape(-1, 1), my_model_data[:,1], test_size=0.25, random_state=42)

In [ ]:
#Define Logistic Regression Model Variable
logr = LogisticRegression()

In [ ]:
# Fit model to training data
logr.fit(x_train, y_train)

## Results

When the positive and negative lake datasets are compared, the bias towards the number of negative lakes becomes apparent. When the means number of accesses per lake are compared between the treatments, it is evident that there is little difference between the treatments for the number of accesses per lake. The regression plot reflects what was inferred from comparing the means with no clear pattern between the number of accesses per lake and the presence of zebra/quagga mussels. 

Testing results are suspect with mostly 0's (absences) predicted and no 1's (presence). Accuracy assessment for training/testing data shows ~90.0 %, but let's check the confusion matrix. Confusion matrix indicates 36 false negatives and no true positives.  311 true negatives were also predicted. The logistic regression coefficient (beta) is 1.68 indicating that for each additional boat ramp the probability of occurence of zebra mussels increases at this rate. When we consider the confusion matrix results this is a bit "hand-wavy".

In [ ]:
# Visualize regression plot.
sns.regplot(x=x_train, y=y_train, data= training_data, logistic=True)

In [ ]:
# Predict to testing dataset
predictions = logr.predict(x_test)
predictions

In [ ]:
# Check Accuracy Score when predicting test data
logr.score(x_test, y_test)

In [ ]:
# Create Pretty Confusion Matrix
cm = metrics.confusion_matrix(y_test, predictions)
score = logr.score(x_test, y_test)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(score)
plt.title(all_sample_title, size = 15);

In [ ]:
# How does liklihood of invasion increase with an increase in ramps?
log_odds = logr.coef_
odds = np.exp(log_odds)
print(odds)

In [ ]:
from scipy.stats import norm
from sklearn.linear_model import LogisticRegression

def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se  
    p = (1 - norm.cdf(abs(t))) * 2
    return p

# test p-values

print(logit_pvalue(logr, x_test))



## Use Model to Predict to Other Data

In [ ]:
# create a random test array
my_test = np.arange(0.0, 100.0 , 1).reshape(-1, 1)

In [ ]:
# Use model to predict test data
logr.predict_proba(my_test)

## Conclusions

The results indicate that my null hypothesis was not falsified. This could related to the dataset being heavilly biased with more records for lakes without zebra/quagga mussels.Though my hypothesis/prediction was not upheld, it is important to report these results because it shows that more than just public accesses is required to predict the presence/absence of zebra/quagga mussels. For example, watercraft visits should also be included to better capture the effect of the use of the accesses within the predictive model. Despite not falsifying my null hypothesis, the code provided here can be modified to ask this question for additional NAS and for other states.  This makes the script here useful beyond the scope of the example analysis.

## References

Fuller P, Neilson M (2015) The U.S. Geological Survey’s Nonindigenous Aquatic Species Database: over thirty years of tracking introduced aquatic species in the United States (and counting). Management of Biological Invasions 6: 159–170. Available from: http://search.proquest.com/docview/2287069464/.

Johnson, L. E., Ricciardi, A., & Carlton, J. T. (2001). Overland dispersal of aquatic invasive species: a risk assessment of transient recreational boating. Ecological applications, 11(6), 1789-1799.

Nelson, N.M. (2019). Enumeration of potential economic costs of dreissenid mussels infestation in Montana. Universtiy of Montana http://dnrc.mt.gov/divisions/cardd/docs/misacdocs/dnrc_economic_cost_dreisseid_mussels_0119.pdf.

Pimentel D, Zuniga R, Morrison D (2005) Update on the environmental and economic costs associated with alien-invasive species in the United States. Ecological Economics 52: 273–288.

Sepulveda AJ, Nelson NM, Jerde CL, Luikart G (2020) Are Environmental DNA Methods Ready for Aquatic Invasive Species Management? Trends in Ecology & Evolution 35: 668–678. https://doi.org/https://doi.org/10.1016/j.tree.2020.03.011 

Wilson G, Aruliah DA, Brown CT, Chue Hong NP, Davis M, Guy RT, et al. (2014) Best Practices for Scientific Computing. PLoS Biol 12(1): e1001745. https://doi.org/10.1371/journal.pbio.1001745